In [1]:
import pandas as pd
from pandas import json_normalize
import requests
import json
import csv
import os
from quant_preprocess import query_and_preprocess_data
from quant_preprocess import query_and_reshape_long
from quant_preprocess import recode_long_data

###notes:
# have done indexes0:3, salt(find the index), and 200:296

params = {
    'api_key':'4F60666AD7F14FE49237DE1B9E2FB925',
    'type':'search',
    'sort_by': 'best_seller'
}


OUT_DIR = "data"
JSON_PATH = os.path.join(OUT_DIR, 'items.txt')
OUT_PATH = os.path.join(OUT_DIR, 'ingredient_prices.csv')

In [2]:
def recoded_to_list():
    df = query_and_reshape_long()
    recoded = recode_long_data(df)
    summary = recoded[["ingredient", "amount"]].groupby("ingredient").agg(["mean", "sum"])
    return summary.index.values.tolist()

In [5]:
def load_ingredients(path):
    """ Loads the list of ingredients to a dataframe """

    ingredients = pd.read_csv(INGRED_PATH, header = None)[200:296]
    return ingredients[0].values.tolist()
def get_item_jsons(params):
    
    json_list = []
    ingredients = load_ingredients(INGRED_PATH)
    for i in ingredients:
        params['search_term'] = i
        result = requests.get('https://api.bluecartapi.com/request', 
                                      params)
        json_list.append(json.dumps(result.json()))
    return json_list
        

In [2]:
def read_txt_as_json():
    
    
    j_list = []
    
    with open("..//data//item_jsons.txt") as text:
        
        json_list = [line.rstrip("\n") for line in text]

        for j in json_list:
            j_list.append(json.loads(j))
        
        return j_list


            
            

In [68]:
l = read_txt_as_json()
l["request_info"]["lemonade"]

TypeError: list indices must be integers or slices, not str

In [66]:
 def get_product_info():
        
        j_list = read_txt_as_json()
        product_dict = {}
        measurements = ["oz", "ml", "lb", "l", "ounces"]
        numbers = ["1", "2", "3", "4", "5", "6", "7", "8", "9","0", "."]
    
        for j in j_list:

            amount_temp = []
            units_temp = []
            price_temp = []
            l = []
            ingredient = j["request_parameters"]["search_term"]
            key = ingredient
            product_dict.setdefault(key, [])
            a = "search_results"
            
            if a not in j:
                continue
            else:
                results = j["search_results"]

                for r in results[0:5]:

                    numbers_temp = []
                    title = r["product"]["title"]
                    if (ingredient in title.lower()):

                        price = r["offers"]["primary"]["price"]
                        price_temp.append(price)

                        for m in measurements:
                            if m in title.lower():
                                units_temp.append(m.strip())
                                break
                            else:
                                continue
                        for t in title[-15:-1]:
                            for n in numbers:
                                if t == n:
                                    numbers_temp.append(n)
                                else:
                                    continue

                        amount_temp.append(''.join(numbers_temp))
                
                        

                product_dict[key].append(price_temp)
                product_dict[key].append(amount_temp)
                product_dict[key].append(units_temp)


                print(price_temp)
                print(units_temp)
                print(amount_temp)

                print(product_dict)
        return product_dict


In [67]:
get_product_info()

[10.98, 5.98, 8.98]
['l', 'ml', 'ml']
['1.75', '375', '375']
{'vodka': [[10.98, 5.98, 8.98], ['1.75', '375', '375'], ['l', 'ml', 'ml']]}
[10.48, 7.82, 13.7, 12.98]
['l', 'l', 'l', 'ml']
['', '30', '', '750']
{'vodka': [[10.98, 5.98, 8.98], ['1.75', '375', '375'], ['l', 'ml', 'ml']], 'gin': [], 'rum': [[10.48, 7.82, 13.7, 12.98], ['', '30', '', '750'], ['l', 'l', 'l', 'ml']]}
[1.12, 0.48, 1.12, 0.48, 1.88]
['l', 'oz', 'l', 'oz', 'oz']
['26', '26', '26', '26', '2.12']
{'vodka': [[10.98, 5.98, 8.98], ['1.75', '375', '375'], ['l', 'ml', 'ml']], 'gin': [], 'rum': [[10.48, 7.82, 13.7, 12.98], ['', '30', '', '750'], ['l', 'l', 'l', 'ml']], 'salt': [[1.12, 0.48, 1.12, 0.48, 1.88], ['26', '26', '26', '26', '2.12'], ['l', 'oz', 'l', 'oz', 'oz']]}
[3.24, 22.99, 29.98, 22.96, 17.14]
['oz', 'oz', 'oz', 'ml', 'oz']
['12', '25', '12', '750', '33.8']
{'vodka': [[10.98, 5.98, 8.98], ['1.75', '375', '375'], ['l', 'ml', 'ml']], 'gin': [], 'rum': [[10.48, 7.82, 13.7, 12.98], ['', '30', '', '750'], ['l', '

KeyError: 'request_parameters'

In [114]:
# def json_to_text(jsons, path):
    
    
#     f = open(path, "w+")
#     for i in jsons:
#         f.write(i + "\n")
#     f.close()
# def to_csv(price_dict, path):
# """ Takes the list of (ingredient, price/oz) tuples and writes them to a csv"""

#     with open(OUT_PATH, 'w+') as out_file:
#         csv_writer = csv.writer(out_file)
#         header = ['ingredient', 'price_per_oz']
#         csv_writer.writerow(header)
#         csv_writer.writerows(price_dict)

In [127]:
# if __name__ == '__main__':

#     os.makedirs(OUT_DIR, exist_ok = True)
#     json_to_text(jsons, JSON_PATH)
#     to_csv(item_dict, OUT_PATH)

    